<a href="https://colab.research.google.com/github/CintiaYYS/DataScience/blob/main/CintiaYurieYamada_SolucaoDeCaso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook utilizado para desenvolver o projeto de solução de estudo de caso para seleção de uma vaga de analista de dados Júnior.

Discussão sobre as problemáticas do caso:

* A tabela de atendimento, que será utilizada para a análise, está no arquivo case_bi_2(2).pdf. Para automatizar o processo é preciso fazer a leitura da tabela. Assim, a linguagem Python não possui uma função que faça esta função de forma direta, por isso, é preciso instalar a biblioteca tabula-py para ler o arquivo pdf e a tabela que está dentro do arquivo.

* Após a leitura da tabela, é preciso verificar como estão os dados. A biblioteca Pandas é utilizada para trabalhar com dataframes. Por isso, é preciso instalá-la.

* Após fazer esses processos de carregar os dados (ETL - Extraction), pode ser feita a parte de transformação (ETL - Tranformation).

* Para fazer a análise, é preciso calcular as seguintes métricas: FCR (First Call Resolution), FDR (First Day Resolution), SLA (Service Level Agreement), taxa de ocupação e outros cálculos.

* Por fim, fazer um relatório para comparar e salvar em um arquivo PDF (ETL - Loading).

#Instalações

In [ ]:
!pip install pandas

In [ ]:
!pip install -q tabula-py

In [ ]:
!pip install fpdf

# Import

Importando as bibliotecas para extração e manipulação dos dados.

In [ ]:
from tabula import read_pdf
import pandas as pd
from fpdf import FPDF

#Carregando os dados obtidos no arquivo case_bi_2_(2).pdf

Como a tabela de ligações está dividida em duas páginas, uma parte com a identificação das colunas e ourta não, é preciso fazer a junção destas duas partes.

Além de verificar se os dados estão corretos. Uma vez que os dados estão em 3 colunas: 0 - composta por texto, 1 - composta pela identificação do dia da ligação e 2 - ID do incidente.

In [ ]:
#Nome do arquivo a ser lido
nome_arq = 'case_bi_2_(2).pdf'

In [ ]:
#Abrindo o arquivo em forma de tabela
tabelas = read_pdf(nome_arq, pages='all',pandas_options={'header':None},stream=True)

## Entendendo o arquivo carregado - como está estruturado, tipo, como iterar e extrair os dados no formato desejado

In [ ]:
print("Tipo da tabela: ",type(tabelas))
print("Tamanho: ",len(tabelas))

Tipo da tabela:  <class 'list'>
Tamanho:  2


### Sabendo que o retorno é do tipo list e que possui tamanho 2, agora é possível verificar o que cada item representa e se esta estrutura pode ser utilizada para as transformações necessárias.

In [ ]:
for tabela in tabelas:
  print("-----------------")
  print(tabela)

-----------------
                                                    0               1  \
0                                                 NaN  Dia da ligação   
1                          Considere o caso fictício:           Dia 2   
2                                                 NaN           Dia 4   
3   Os clientes de uma empresa podem ligar para tr...             NaN   
4                                                 NaN           Dia 1   
..                                                ...             ...   
58                                                NaN           Dia 5   
59                                                NaN           Dia 6   
60                                                NaN           Dia 6   
61                                                NaN           Dia 2   
62                                                NaN           Dia 2   

                  2  
0   ID de Incidente  
1                 1  
2                 1  
3               N

# Tratando os dados para manipulação

### Na primeira página há 3 colunas: uma de texto com as Considerações sobre o caso fictício, uma que identifica o dia da ligação e outro que identifica o incidente.
Já na segunda página do PDF há apenas duas colunas: identificação do dia da ligação e outra com a identificação do incidente.

Assim, é preciso estruturar o código para automtizar a extração de forma correta.

In [ ]:
pag1 = tabelas[0].iloc[:,[1,2]]
pag1

,1,2
0,Dia da ligação,ID de Incidente
1,Dia 2,1
2,Dia 4,1
3,NaN,NaN
4,Dia 1,2
...,...,...
58,Dia 5,21
59,Dia 6,22
60,Dia 6,22
61,Dia 2,24


In [ ]:
#Eliminando as linhas que possuem NaN por conta do texto do PDF ("Considere o caso fictício: ....")
pag1 = pag1.dropna()
pag1

,1,2
0,Dia da ligação,ID de Incidente
1,Dia 2,1
2,Dia 4,1
4,Dia 1,2
6,Dia 7,2
7,Dia 1,3
8,Dia 6,3
9,Dia 2,3
10,Dia 3,3
11,Dia 1,3


É preciso que o índice das colunas sejam 0 e 1 para que a concatenação das tabelas das páginas ocorram de forma correta, para ter correspondência.

In [ ]:
pag1.columns=[0,1]
pag1

,0,1
0,Dia da ligação,ID de Incidente
1,Dia 2,1
2,Dia 4,1
4,Dia 1,2
6,Dia 7,2
7,Dia 1,3
8,Dia 6,3
9,Dia 2,3
10,Dia 3,3
11,Dia 1,3


In [ ]:
pag2 = tabelas[1]
pag2

,0,1
0,Dia 2,25
1,Dia 2,26
2,Dia 5,27
3,Dia 3,27
4,Dia 7,27
5,Dia 2,27
6,Dia 1,28
7,Dia 3,28
8,Dia 6,28
9,Dia 3,29


### Unindo as tabelas

In [ ]:
atendimento = pd.concat([pag1,pag2],ignore_index=True)
atendimento

,0,1
0,Dia da ligação,ID de Incidente
1,Dia 2,1
2,Dia 4,1
3,Dia 1,2
4,Dia 7,2
...,...,...
100,Dia 3,48
101,Dia 4,48
102,Dia 2,49
103,Dia 3,50


## Agora que os dados estão em um dataframe, é preciso fazer algumas modificações:

* A primeira linha é a identificação da coluna;
* Verificar se há valores NaN no índice e no dia da ligação.

### Identificando a primeira linha como cabeçalho do dataframe, e atualizando tabela a partir da linha 1, pois a linha 0 é a identificação da coluna

In [ ]:
atendimento.columns = atendimento.iloc[0]
atendimento = atendimento[1:].reset_index(drop=True)
atendimento

,Dia da ligação,ID de Incidente
0,Dia 2,1
1,Dia 4,1
2,Dia 1,2
3,Dia 7,2
4,Dia 1,3
...,...,...
99,Dia 3,48
100,Dia 4,48
101,Dia 2,49
102,Dia 3,50


### Verificando se há valores null ou NaN

In [ ]:
for coluna in atendimento.columns:
  print("Coluna: ",coluna)
  print("Contém nulo: ",atendimento[coluna].isnull().any())
  print("Contém NaN: ",atendimento[coluna].isna().any())


Coluna:  Dia da ligação
Contém nulo:  False
Contém NaN:  False
Coluna:  ID de Incidente
Contém nulo:  False
Contém NaN:  False


## Para calcular a quantidade de dias que cada chamado levou para ser resolvido, é preciso utilizar apenas o número que identifica o Dia da ligação, assim, a coluna Dia da ligação passará a ser apenas o inteiro que identifica o dia.

Na origem, Dia da ligação é uma string, após fazer a separação, os números não são inteiros e sim string, por isso, qdo aplicada a função lambda em cada linha, já é feita a conversão de tipo.

In [ ]:
atendimento['Dia da ligação'] = atendimento['Dia da ligação'].str.split(" ").apply(lambda x:int(x[1]))
atendimento

,Dia da ligação,ID de Incidente
0,2,1
1,4,1
2,1,2
3,7,2
4,1,3
...,...,...
99,3,48
100,4,48
101,2,49
102,3,50


In [ ]:
#Verificando o tipo do Dia da ligação
type(atendimento['Dia da ligação'][0])

numpy.int64

# Conhecendo os valores de cada coluna

In [ ]:
atendimento['Dia da ligação'].unique()

array([2, 4, 1, 7, 6, 3, 5])

In [ ]:
len(atendimento['Dia da ligação'].unique())

7

In [ ]:
atendimento['ID de Incidente'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '20', '21', '22', '24', 25, 26, 27, 28, 29,
       30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50], dtype=object)

In [ ]:
len(atendimento['ID de Incidente'].unique())

44

In [ ]:
#Criando uma variável auxiliar que vai armazenar o texto do relatório.
relatorio = " RELATÓRIO DE LIGAÇÕES PARA TRATAR INCIDENTES \n\n\n"
relatorio += "Relatório referente a 7 dias de atendimento e 44 incidentes.\n\n"

É possível verificar que são 7 dias (1 a 7) de ligação e 44 id de incidentes (1 a 50, sendo que alguns identificadores não estão registrados).

Lembrando que o dia da ligação é identificado por uma String "Dia x"

# Fazendo os cálculos para as métricas

## Métricas

O incidente é classificado como FCR se o ID de Incidente tem apenas um dia de ligação e apenas um resgitro de ligação. Ou seja, na tabela, há apenas uma linha com o ID de Incidente.

A cada ocorrência de FCR, será adicionado ao contador, variável definida como FCR.

Identificando um FDR: quando o incidente tem apenas duas linhas e as duas tem o mesmo dia de ligação.

O SLA será calculado em duas etapas: somar a quantidade de dias que cada incidente demorou para ser resolvido e em seguida, dividir esta soma pela quantidade de incidentes, obtendo-se assim a média em dias.

In [ ]:
FCR = 0
FDR = 0
total_dias = 0
for incidente in atendimento['ID de Incidente'].unique():
  if len(atendimento[atendimento['ID de Incidente']==incidente]) == 1:
    FCR += 1
  else:
     if len(atendimento[atendimento['ID de Incidente']==incidente]['Dia da ligação'].unique()) == 1:
      FDR += 1
  inicio = atendimento[atendimento['ID de Incidente']==incidente]['Dia da ligação'].unique().min()
  final = atendimento[atendimento['ID de Incidente']==incidente]['Dia da ligação'].unique().max()
  dias_para_resolver = final - inicio + 1
  total_dias += dias_para_resolver

SLA = total_dias/len(atendimento['ID de Incidente'].unique())
print("FCR: ",FCR)
print("FDR: ",FDR)
print("SLA: {0:.0f}".format(SLA))


FCR:  9
FDR:  6
SLA: 3


In [ ]:
relatorio += "Métricas \n \nNível se serviço mínimo esperado é de {0:.0f} dias.\n".format(SLA)

## Definindo as porcentagens de FCR e FDR, e a média de dias que cada incidente demora para ser resolvido.

In [ ]:
FCR_porcentagem = (FCR/len(atendimento['ID de Incidente'].unique()))*100
FDR_porcentagem = (FDR/len(atendimento['ID de Incidente'].unique()))*100

print("FCR %: {0:.2f}%".format(FCR_porcentagem))
print("FDR %: {0:.2f}%".format(FDR_porcentagem))

FCR %: 20.45%
FDR %: 13.64%


In [ ]:
relatorio += "FCR: {0:.2f}%\n".format(FCR_porcentagem)
relatorio += "FDR: {0:.2f}%\n\n".format(FDR_porcentagem)

# Prévia sobre as métricas:

A meta é alcançar FCR em 20% dos incidentes e FDR em 30% dos incidentes e respeitar um SLA de 2 dias para resolução dos incidentes.

Porém, a atual situação é:
* FCR em 20.45% - meta alcançada
* FDR em 13.64% - meta não alcançada
* SLA de 3 dias - meta não alcançada

In [ ]:
relatorio += "Metas \n \n FCR em 20% \n FDR em 30% \n SLA de 2 dias \n \n "
relatorio += "Apenas a meta de FCR foi atingida. \n\n"

# Calculando o desempenho de cada atendente da tabela atual

Um mesmo atendente pode atender duas ligações e tratá-las até o final, porém a carga horária do atendente é limitada a 5 horas por dia, foram 7 dias de trabalho e cada atendimento leva em média 20 minutos.

* 1- Calcular quantas horas de atendimento foram necessárias para os 7 dias.
* 2- Verificar o limite de atendimentos por dia/atendente para saber quantos atendentes precisam para atender os incidentes da tabela (como na tabela não há a hora do atendimento, será simulado que o atendente recebe ligação a partir do momento zero e pode resolver até o último minuto do horário de serviço).
* 3- Verificar a quantidade máxima de ligações realizados no mesmo dia.

In [ ]:
# Unidade de medida é minuto
tempo_atendimento = 20
tempo_por_dia = 5*60
tempo_por_semana = 7*tempo_por_dia

limite_atendimento_por_dia = tempo_por_dia/tempo_atendimento
limite_atendimento_por_dia

15.0

## É preciso selecionar quantas ligações foram feitas em cada dia, sejam FCR, FDR ou incidentes que demoraram mais de um dia para resolver.

In [ ]:
atendimento_por_dia = []
for dia in atendimento['Dia da ligação'].unique():
  atendimento_por_dia.append([dia,len(atendimento[atendimento['Dia da ligação']==dia])])

atendimento_por_dia = sorted(atendimento_por_dia,key=lambda x:x[1],reverse=True)

for d in atendimento_por_dia:
  print("Dia ",d[0]," teve ",d[1]," atendimentos.")
  relatorio+="Dia {:d} teve {:d} atendimentos.\n".format(d[0],d[1])

Dia  2  teve  24  atendimentos.
Dia  3  teve  21  atendimentos.
Dia  6  teve  14  atendimentos.
Dia  5  teve  14  atendimentos.
Dia  4  teve  11  atendimentos.
Dia  7  teve  11  atendimentos.
Dia  1  teve  9  atendimentos.


In [ ]:
relatorio += "Contratação de Atendentes \n\n"
relatorio += "Com base na duração média de cada ligação e no tempo de trabalho diário de um atendente, o limite de ligações que um atendente consegue atender é de {0:.0f}.\n".format(limite_atendimento_por_dia)
relatorio += "Assim, para atender os incidentes dada a tabela, são necessários 2 atendentes.\n\n"

## O dia com mais ligações recebidas foi Dia 2, com 24 atendimentos.

Como um atendente consegue atender até 15 ligações por dia, 2 atendentes conseguem suprir a demanda de ligações da tabela.

# Calculando a taxa de ocupação

É o percentual de horas que um atendente passa atendendo efetivamente / todas as horas trabalhadas

Lembrando que a quantidade de atendimento é igual a quantidade de registros na tabela

In [ ]:
tempo_atendendo = tempo_atendimento * len(atendimento)
tempo_atendendo

2080

# Para definir a taxa de ocupação, é preciso calcular quantos minutos de ligação a empresa é capaz de atender.

Como a empresa deve contratar 2 atendentes para suprir às necessidades dos incidentes da tabela, a taxa deve ser proporcional a quantidade que os dois atendentes podem atender.

Ou seja,
Taxa de ocupação = Tempo que os incidentes exigiram / tempo que todos os atendentes conseguem trabalhar .

In [ ]:
num_atendentes = 2
taxa_ocupacao = (tempo_atendendo/(tempo_por_semana * num_atendentes))*100
print("A taxa de ocupação é de {0:.2f}%".format(taxa_ocupacao))

A taxa de ocupação é de 49.52%


In [ ]:
relatorio += "A taxa de ocupação é de {0:.2f}%.\n\n".format(taxa_ocupacao)

# Considerações gerais

In [ ]:
relatorio += "CONSIDERAÇÕES GERAIS \n\n\n"
relatorio += "Foram 3 metas estipuladas pela empresa, porém apenas uma foi atingida, a quantidade de FCR. \n"
relatorio += "Para as demais metas, FDR e SLA de 2 dias, é preciso fazer alguns ajustes para melhorar o tempo para resolver um incidente.\n"
relatorio += "Dois atendentes são suficientes para atender a quantidade de ligações, baseada na quantidade da tabela.\n"
relatorio += "Dado que os atendentes conseguem atender as ligações, é preciso verificar outras informações sobre os incidentes com o objetivo de melhorar o serviço e tempo de resolução de incidentes."

# Gerando o arquivo no formato PDF com as informações básicas que foram extraídas da análise dos dados.

## Criando uma instância FPDF que será utlizada para configurar e gerar o arquivo PDF

In [ ]:
pdf = FPDF()

## Adicionando a página com as informações

In [ ]:
pdf.add_page()
pdf.set_font("Arial",size=14)
pdf.multi_cell(0,10,relatorio)

[]

## Salvando o arquivo

In [ ]:
pdf.output("RelatorioIncidentes.pdf")

''